# Q1 Implementing a Neural Network

Hey hi 🫠. In this exercise, you will be using this [dataset](https://www.kaggle.com/datasets/emirhanai/social-media-usage-and-emotional-well-being?select=val.csv&fbclid=IwAR0f2phm0US5CPA62eGmVrXSBsirYWkziveSWYqdPVcAx9cbGebmr6gtp3k) (We already downloaded it for you so you don't have to do that again). Your tasks are:
- Preprocess the dataset, including splitting into input features and labels as well as handling null values and biases
- Implement a fully-connected neural network with 2 layers (1 hidden layer) to predict people's well-being based on different features

Let's start by importing the neccessary modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

Load the data from folder `/dataset`

In [ ]:
file_path = './datasets'

# If you are running from Google Colab, uncomment the section below
# # Mount google drive to allow access to your files
# from google.colab import drive
# drive.mount('/content/drive')
# drive_folder = '/content/drive/MyDrive'

# # Ajust this line to be the assignment2 folder in your google drive
# notebook_folder = drive_folder + '/assignment2'
# file_path = notebook_folder + file_path

In [ ]:
df_train = pd.read_csv(file_path + '/train.csv')

df_train.info()

Peeking at the data, we see that the label feature is **Dominant_Emotion** and our input features are **User_ID**, **Age**, **Platform**, ..., **Message_Sent_Per_Day**. We will be training our model by our input features to predict the entries' labels.

So first, we will split into features x and label y

In [ ]:
# Target variable (y)
df_train_y = df_train[['Dominant_Emotion']]

# Features (X)
df_train_x = df_train.drop('Dominant_Emotion', axis=1)

print(df_train_x.shape)
print(df_train_y.shape)

## Data Preprocessing

### 1. Remove unnecessary features

Most features as we see above can all have an impact on that person's well-being. However, there is one feature that seems to not have an impact at all. Can you find which feature is it? If you can, please also remove it in the code block below!

In [ ]:
#TODO: Remove the unnecessary feature

# Display all columns
df_train_x.info()

### 2. Filling Missing Entries

The dataset is still raw, but why does it matter?\
\
In the course of data collection, the data collectors might have not filled in all the columns for some entries, causing those entries to contain null values. Then, if we feed null values to our model, what would happen? Your computer will explode.\
\
Just kidding of course, but it will definitely throw some errors.\
\
This is where preprocessing come in. Let's first handle the missing value. But where are all of the missing values? We could find this by:

In [ ]:
missing_values = df_train_x.isnull().any()
print(missing_values)

So, sadly speaking, almost every column have missing entries (we just don't know how many for now, but of course we can check as below!

In [ ]:
# Create a DataFrame with the feature names and the sum of null values
null_counts_x = pd.DataFrame({'Types': df_train_x.dtypes, 'Null Values': df_train_x.isnull().sum()})
null_counts_y = pd.DataFrame({'Types': df_train_y.dtypes, 'Null Values': df_train_y.isnull().sum()})

# Print the tables
print("Training set:")
print(null_counts_x)
print("\nTest set:")
print(null_counts_y)

So how can we handle these missing entries? We can fill the missing entries by their **mean** or **median**.

But what about features that aren't numbers? In that case, we can fill them by their **mode**, the most frequent value of the column

We have filled out for one column as an example. Please fill in the rest:

In [ ]:
%%capture --no-stdout

def handle_null(train_x, train_y):
    """
    Preprocesses the training data by handling missing entries in the features.

    Args:
        train_x (DataFrame): The training data features.
        train_y (DataFrame): The training data labels.

    Returns:
        None
    """
    train_x["Daily_Usage_Time (minutes)"].fillna(train_x["Daily_Usage_Time (minutes)"].mode()[0], inplace=True)

    # TODO: Handle missing entries for the remaining features in the training set
    pass

    # TODO: Handle missing entries for the "Dominant_Emotion" feature in the test set

# Fill in null values and display data
handle_null(df_train_x, df_train_y)

null_counts_x = pd.DataFrame({'Types': df_train_x.dtypes, 'Null Values': df_train_x.isnull().sum()})
null_counts_y = pd.DataFrame({'Types': df_train_y.dtypes, 'Null Values': df_train_y.isnull().sum()})

print("Training set:")
print(null_counts_x)
print("\nTest set:")
print(null_counts_y)

Noticed how after filling data, we no longer have missing entries.\
\
There are a few ways to deal with missing entries:
- If there are not much missing entries, we can filling by **mean**, **mode**, **meadian**, **interpolation**, etc.
- If there are too much missing entries for a feature, we can consider remove that feature

### 3. Handling String Data

Moving on, noticed class **Gender**, **Platform**, and **Dominant_Emotion** is now denote in form of *string* - which is something that computer isn't really good at processing. Everything comes in form of *number* will makes computer happy to process and to work with.

One of the way that everyone's immediately come up with is by assigning a number to each of the class, for instance:
$$
    \text{Hapiness} = 1 \qquad
    \text{Neutral} = 2 \qquad
    \text{Anger} = 3 \qquad
    \text{Anxiety} = 4 \qquad
    \text{Boredom} = 5 \qquad
$$
There is a problem. We are giving each emotion a bias. As above, the model can interpret "Anger" as being closer to "Happiness" than "Boredom", which isn't necessarily true is it?

Already, this creates a hazy line not only for us but for any model that we are about to train because again, computers can only understand numbers. Different numbers (weight of the features) can affect the importance of each feature through the model's view and thus result in inaccurate predictions.

So how do we overcome this problem? One of the way is to use one-hot encoding🔥, which is a technique for removing bias by splitting differnt classes into independent features. Below is what would look like if we apply one-hot encoding on the emotions:
$$
    \text{Hapiness} = \begin{bmatrix}
        1 \\
        0 \\
        0 \\
        0 \\
        0
    \end{bmatrix} \space
    \text{Neutral} = \begin{bmatrix}
        0 \\
        1 \\
        0 \\
        0 \\
        0
    \end{bmatrix} \space
    \text{Anger} = \begin{bmatrix}
        0 \\
        0 \\
        1 \\
        0 \\
        0
    \end{bmatrix} \space
    \text{Anxiety} = \begin{bmatrix}
        0 \\
        0 \\
        0 \\
        1 \\
        0
    \end{bmatrix} \space
    \text{Boredom} = \begin{bmatrix}
        0 \\
        0 \\
        0 \\
        0 \\
        1
    \end{bmatrix} \space
$$

What we are doing here is that we are just showing the existence of the feature $1$ denote **exist** and $0$ mean **does not exist**.

We will use this to remove biases from features like "Dominant_Emotion", as well as "Gender" and "Platform".

In [ ]:
df_train_x = pd.get_dummies(df_train_x, columns=['Gender', 'Platform'], dtype=int)
df_train_y = pd.get_dummies(df_train_y, columns=['Dominant_Emotion'], dtype=int)

df_train_x.info()

In [ ]:
df_train_y.info()

### 3. Conversion To Numpy

Simple as it is, `to_numpy()` will help handle that

In [ ]:
x_train = df_train_x.to_numpy()
y_train = df_train_y.to_numpy()

print(x_train.shape)
print(y_train.shape)

## Model Implementation

### 1. Initalize random weights and biases

First, let's create two sets of random weights and zero biases for our input layer and hidden layer.

In [ ]:
hidden_size = 5

# Initialize the weights and biases
def init():
    ## TODO: Initialize W1, b1, W2, b2
    pass

### 2. Activation Functions

When using along with neural network, activation functions introduce non-linear properties to the network which helps increase accuracy for complex classification problems such as this one. We will be using 3 types of activation functions and compare each of them:

- Sigmoid: 
$$
    \begin{align*}
        f(x) &= \sigma(x) = 1/(1+e^{-x}) \\
        f'(x) &= \sigma(x) * (1 - \sigma(x))
    \end{align*}
$$

- Tanh:
$$
    \begin{align*}
        f(x) &= \tanh(x) \\
        f'(x) &= 1 - \tanh^2(x)
    \end{align*}
$$

- ReLU:
$$
    \begin{align*}
        f(x) &= \text{max}(0, x) \\
        f'(x) &= 
        \begin{cases}
            0, & \text{if }\, x < 0 \\
            1, & \text{if }\, x > 0
        \end{cases}
    \end{align*}
$$

Based on the formulas above, implement the activation functions along with their analytical gradient below using numpy:

In [ ]:
def sigmoid(x, is_grad=False):
    ## TODO: Add your implementation here
    pass

def tanh(x, is_grad=False):
    ## TODO: Add your implementation here
    pass

def relu(x, is_grad=False):
    ## TODO: Add your implementation here
    pass

### 3. Forward Pass

This function will pass the input through the network from the input layer to the output layer to compute the network's predictions by applying the activation function to the hidden layer and compute the score $f(x, W)$.

In [ ]:
# Pass input data through a hidden layer given sets of weights and biases and return the output
def forward_pass(f, x, W1, b1, W2, b2):
    """
    Perform a forward pass through the neural network.

    Args:
        f: Activation function to be applied to the hidden layer and output layer.
        x: Input data.
        W1: Weight matrix for the first layer.
        b1: Bias vector for the first layer.
        W2: Weight matrix for the second layer.
        b2: Bias vector for the second layer.

    Returns:
        scores: Output scores after the forward pass.

    """
    ## TODO: Add your implementation here
    pass
    

### 4. Backward Pass

Now that we have the output, let's compare with our label y and compute the loss using Softmax. Make sure you apply L2 regularization at the end. The loss function should follow the formula below:
$$
    \begin{align*}
        L_i(s, y_i) &= -\log \left(\frac{e^{s_{y_i}}}{\sum_j e^{s_j}}\right) \\
        L &= \frac{1}{N} \sum^N_{i=1} L_i(s, y_i) + \lambda R(W) \\
        R(W) &= \sum_k \sum_l W^2_{k,l}
    \end{align*}
$$

After computing the loss, we will propagate backward to update the weights and biases using the activation function's derivate (the change in loss in respect to change in weight or biases).

In [ ]:
def backward_pass(f, x, y, scores, W1, b1, W2, b2, lam=0.0):
    """
    Compute the loss function for a given set of scores and labels and pass backward.

    Parameters:
        f (numpy.ndarray): Hidden layer activations after the forward pass.
        x (numpy.ndarray): The input data, shape (N, D).
        y (numpy.ndarray): The ground truth labels, shape (N, C).
        scores (numpy.ndarray): The predicted scores for each class, shape (N, C).
        lam (float, optional): The regularization parameter. Default is 0.0.

    Returns:
        tuple: A tuple containing the computed loss value and the gradients.

        - loss (float): The computed loss value.
        - grads (dict): Dictionary mapping elements to their gradient in respect to the loss function (W1, b1, W2, b2)
    """
    ## TODO: Add your implementation here
    pass

### 5. Train the model

Now we will start training the model using gradient descent.

In [ ]:
def predict(f, x, W1, b1, W2, b2):
    """
    Predicts the class labels for the given input data using a neural network model.

    Parameters:
    f (function): The activation function to be used in the neural network.
    x (numpy.ndarray): The input data of shape (N, D), where N is the number of samples and D is the number of features.
    W1 (numpy.ndarray): The weight matrix of the first layer of the neural network.
    b1 (numpy.ndarray): The bias vector of the first layer of the neural network.
    W2 (numpy.ndarray): The weight matrix of the second layer of the neural network.
    b2 (numpy.ndarray): The bias vector of the second layer of the neural network.

    Returns:
    numpy.ndarray: The predicted class labels for the input data, of shape (N,).
    """
    scores = forward_pass(f, x, W1, b1, W2, b2)
    y_pred = np.argmax(scores, axis=1)
    return y_pred

def train(f, x_train, y_train, W1, b1, W2, b2, lam=5e-6, learning_rate=0.001, num_iters=100):
    """
    Trains a neural network model using the specified parameters.

    Parameters:
    - f (function): The activation function to use in the neural network.
    - x_train (numpy.ndarray): The input training data.
    - y_train (numpy.ndarray): The target training data.
    - W1 (numpy.ndarray): The weights of the first layer.
    - b1 (numpy.ndarray): The biases of the first layer.
    - W2 (numpy.ndarray): The weights of the second layer.
    - b2 (numpy.ndarray): The biases of the second layer.
    - lam (float): The regularization parameter (default: 5e-6).
    - learning_rate (float): The learning rate for gradient descent (default: 0.001).
    - num_iters (int): The number of iterations for training (default: 100).

    Returns:
    - W1 (numpy.ndarray): The updated weights of the first layer.
    - b1 (numpy.ndarray): The updated biases of the first layer.
    - W2 (numpy.ndarray): The updated weights of the second layer.
    - b2 (numpy.ndarray): The updated biases of the second layer.
    - loss_history (list): The history of loss values during training.
    - acc_history (list): The history of accuracy values during training.
    """
    loss_history = []
    acc_history = []

    ## TODO: Add your implementation here
    pass                                                                                                                                                 

### 6. Putting it all together

#### Train with sigmoid

In [ ]:
W1, b1, W2, b2 = init()
W1_trained_sig, b1_trained_sig, W2_trained_sig, b2_trained_sig, loss_history_sig, acc_history_sig = train(sigmoid, x_train, y_train, W1, b1, W2, b2)
print("No error for sigmoid! Amazing job!")

##### Loss history

In [ ]:
def history_plot(history, ylabel):
    plt.figure()
    plt.plot(np.arange(1, len(history) + 1), history, marker=".")
    plt.xlabel("iterations")
    plt.ylabel(ylabel)
    plt.show()

In [ ]:
history_plot(loss_history_sig, "Sigmoid loss")

##### Accuracy history

In [ ]:
history_plot(acc_history_sig, "Sigmoid accuracy")

### Training with tanh

In [ ]:
W1, b1, W2, b2 = init()
W1_trained_tanh, b1_trained_tanh, W2_trained_tanh, b2_trained_tanh, loss_history_tanh, acc_history_tanh = train(tanh, x_train, y_train, W1, b1, W2, b2)
print("No error for tanh! Amazing job!")

##### Loss history

In [ ]:
history_plot(loss_history_tanh, "Tanh loss")

##### Accuracy history

In [ ]:
history_plot(acc_history_tanh, "Tanh accuracy")

#### Training with ReLU

In [ ]:
W1, b1, W2, b2 = init()
W1_trained_relu, b1_trained_relu, W2_trained_relu, b2_trained_relu, loss_history_relu, acc_history_relu = train(relu, x_train, y_train, W1, b1, W2, b2)
print("No error for ReLU! Amazing job!")

##### Loss history

In [ ]:
history_plot(loss_history_relu, "ReLU loss")

##### Accuracy history

In [ ]:
history_plot(acc_history_relu, "ReLU accuracy")

## Testing

Now we will now use the model to predict the label of the entries in our test set

Again, let's handle the test set

In [ ]:
df_test = pd.read_csv(file_path + '/test.csv')
df_test = df_test.drop('User_ID', axis=1)

df_test.info()

In [ ]:
# Target variable (y)
df_test_y = df_test[['Dominant_Emotion']]

# Features (X)
df_test_x = df_test.drop('Dominant_Emotion', axis=1)

print(df_test_x.shape)
print(df_test_y.shape)

In [ ]:
missing_values = df_test_x.isnull().any()
print(missing_values)

Cool thing here, we have no missing value!

In [ ]:
df_test_x = pd.get_dummies(df_test_x, columns=['Gender', 'Platform'], dtype=int)
df_test_y = pd.get_dummies(df_test_y, columns=['Dominant_Emotion'], dtype=int)

df_test_x.info()

In [ ]:
x_test = df_test_x.to_numpy()
y_test = df_test_y.to_numpy()

print(x_test.shape)
print(y_test.shape)

In [ ]:
def compute_accuracy(pred, y):
    ## TODO: Add your implementation here for checking accuracy of the model
    pass

In [ ]:
sigmoid_pred = predict(sigmoid, x_test, W1_trained_sig, b1_trained_sig, W2_trained_sig, b2_trained_sig)
sigmoid_acc = compute_accuracy(sigmoid_pred, y_test)
print(f"Sigmoid accuracy: {sigmoid_acc}")

tanh_pred = predict(tanh, x_test, W1_trained_tanh, b1_trained_tanh, W2_trained_tanh, b2_trained_tanh)
tanh_acc = compute_accuracy(tanh_pred, y_test)
print(f"Tanh accuracy: {tanh_acc}")

relu_pred = predict(relu, x_test, W1_trained_relu, b1_trained_relu, W2_trained_relu, b2_trained_relu)
relu_acc = compute_accuracy(relu_pred, y_test)
print(f"ReLU accuracy: {relu_acc}")

Compare the three results from the activation functions above. Which one is the most effective?

- Your answer:

See if you can tune any parameters to increase your accuracy. You can put in your highest achieved accuracy here with a brief explanation of how you did it! (We might put up a leaderboard on the website 🤔)

- Highest accuracy:
- Method:

Hey hi, feel free to change any functions on "Model implementation". The creativity is unbound! Good luck on the homework!